## Definições
Bibliotecas utilizadas e definições de path. Para facilitar a manipulação de arquivos, o diretório do projeto é definido neste ponto. Também é realizada a carga dos dados que serão utilizados neste repositório. Neste notebook, o conjunto saída do feature_engineering é o principal arquivo de trabalho.

In [13]:
import numpy as np
import pandas as pd
from datetime import datetime
from typing import Tuple, List
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

#base_path = "C:/Users/99818854/Projetos/GitRep/adaptive_learning"
base_path = "/media/bruno/Arquivos/Desenvolvimento/NextQuestion"

In [14]:
base = pd.read_csv(f"{base_path}/data/mastery.csv")

## Estratégia

Uma vez que determinados o modo de calcular a maestria, falta encontrar a pontuação que será atribuída para cada acerto (geral e por dificuldade). Não sabemos ao certo como definir estes valores numéricos, porém eles possuem grande importância, seja para permitir o funcionamento do modelo quanto para interpretar os resultados. Afinal, se encontrarmos um ideal que pontua mais acertos de questões difíceis, podemos inferir que acertar questões difíceis contribui mais para a obtenção de maestria.

Voltando ao ponto principal, de como obter os parâmetros de pontuação que melhor definem a maestria, podemos simplesmente testar diversos valores até achar o que melhor se adequa. É uma abordagem válida, desde que se tenham disponíveis recursos suficientes. Pensando em heurística, vamos nos contentar com um conjunto de valores bom, não necessariamente excelentes, isso já é suficiente para reduzir o tempo de busca.

No lugar de simplemente chutar diversos valores ao acaso, vamos pegar emprestada a abordagem dos algoritmos genéricos, onde testandos um conjunto de parâmetros (população), selecionamos os melhores e realizamos mutações entre eles. A base deste algoritmo é uma função objetivo, resposável por dizer o quão melhor é um conjunto de parâmetros em relação à outro.

Funções objetivos são o grande fino deste tipo de abordagem. Para este caso, vamos utilizar uma regressão logística, que vai receber a maestria calculada e preparar um modelo de aprendizagem de máquina responsável por dizer se, dada uma maestria e dificuldade, o respondente é capaz de acertar uma questão. Das métricas disponíveis em problemas de classificação, vamos utilizar o f1-score como resultado da função objetivo para avaliar o melhor conjunto de parâmetros.

Resumindo, a partir das diversas gerações de populações arbitrárias de conjuntos de parâmetros de pontuação, vamos buscar aquele conjunto que melhor calcula a maestria que torna possível criar uma regressão logística com maior nível de eficiência. É importante pontuar que o objetivo final deste notebook não é encontrar os valores mais corretos, mas sim valores base bons, que permitam o processo de modelagem e interpretação dos resultados.

In [15]:
def objective_function(base: pd.DataFrame, params: np.array) -> Tuple[float]:
    # Acessa apenas as fatias importantes para cálculo de maestria, computando
    # os conjuntos de entrada (X) e resposta (y) separados em treino e teste
    mastery = np.dot(base.values[:, 3:-1], params)
    X, y = np.concatenate((base.values[:, 2:3], mastery), axis=1), base.values[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    # Substitui os valores nulos de dificuldade, necessárias para o modelo funcionar
    mean_difficulty = round(np.nanmean(X_train[:, 0]))
    X_train[np.isnan(X_train)] = mean_difficulty
    X_test[np.isnan(X_test)] = mean_difficulty

    # Execução do treinamento da função logística
    model = SGDClassifier(loss="log_loss", learning_rate="adaptive", eta0=0.001)
    model.fit(X_train, y_train)

    # Computação das métricas de classificação
    y_pred_test = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

    accuracy = (tp + tn) / (tn + fp + fn + tp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)

    return accuracy, precision, recall, f1

In [16]:
def random_params() -> List[float]:
    # ganho | dificuldade 1 | dificuldade 2 | dificuldade 3 | dificuldade 4 | dificuldade 5
    # Por definição, vamos manter o ganho total sempre maior do que os ganhos bônus por dificuldade
    return [
        np.random.random() * 4 + 1,
        np.random.random(),
        np.random.random(),
        np.random.random(),
        np.random.random(),
        np.random.random()
    ]

In [17]:
def evaluate(population: list) -> List[float]:
    results = []
    start_date = datetime.now()
    for i in range(0, len(population)):
        print(f"Avaliando o indivíduo {i + 1}/{len(population)} da população...", end="\r")
        accuracy, precision, recall, f1 = objective_function(base, np.array(population[i]).reshape((len(population[i]), 1)))
        results.append(f1)

    end_date = datetime.now()
    duration = round((end_date - start_date).seconds / 60, 2)
    print(f"Melhor F1-Score encontrado: {f1} ({duration} min)")
    return results

In [18]:
def select(population, results) -> List[float]:
    combination = list(zip(population, results))
    select_1 = combination[np.random.randint(0, len(combination))]
    select_2 = combination[np.random.randint(0, len(combination))]
    return select_1[0] if select_1[1] > select_2[1] else select_2[0]

In [19]:
def crossover(father, mother) -> Tuple[List[float], List[float]]:
    element_1, element_2 = father, mother
    if np.random.randint(0, 100) < 50:
        point = np.random.randint(0, len(father))
        element_1 = father[:point] + mother[point:]
        element_2 = mother[:point] + father[point:]
    return element_1, element_2

In [20]:
def mutate(element):
    for i in range(0, len(element)):
        if np.random.randint(0, 100) < 2:
            # Aplica a mesma regra de negócio da random_params, garantindo um
            # valor maior para o ganho total do que o ganho por dificuldade
            if i == 0:
                element[i] = np.random.random() * 4 + 1
            else:
                element[i] = np.random.random()

In [21]:
# Define e inicializa os parâmetros do algoritmo genético
generations = 10
qntd_individuals = 10
current_population = [random_params() for i in range(0, qntd_individuals)]

print("Geração primitiva:")
current_results = evaluate(current_population)
epochs = [list(zip(current_population, current_results))]

for i in range(0, generations):
    print(f"\nGeração {i + 1}")
    new_population = []

    # Compila uma nova população a partir da geração atual
    while len(new_population) < qntd_individuals:
        father = select(current_population, current_results)
        mother = select(current_population, current_results)
        element_1, element_2 = crossover(father, mother)
        mutate(element_1)
        mutate(element_2)
        new_population.append(element_1)
        new_population.append(element_2)

    # Verifica a eficiência da nova população de acordo com a função objetivo
    current_population = new_population
    current_results = evaluate(current_population)
    epochs.append(list(zip(current_population, current_results)))

Geração primitiva:
Melhor F1-Score encontrado: 0.782815200103661 (1.28 min)

Geração 1
Melhor F1-Score encontrado: 0.7796780297516812 (1.32 min)

Geração 2
Melhor F1-Score encontrado: 0.7821179037291381 (1.33 min)

Geração 3
Melhor F1-Score encontrado: 0.7829737504907577 (1.3 min)

Geração 4
Melhor F1-Score encontrado: 0.777426067483777 (1.35 min)

Geração 5
Melhor F1-Score encontrado: 0.7833173394969793 (1.33 min)

Geração 6
Melhor F1-Score encontrado: 0.7807695340219191 (1.35 min)

Geração 7
Melhor F1-Score encontrado: 0.780273366120561 (1.33 min)

Geração 8
Melhor F1-Score encontrado: 0.782203556663823 (1.35 min)

Geração 9
Melhor F1-Score encontrado: 0.7827858963553883 (1.35 min)

Geração 10
Melhor F1-Score encontrado: 0.776443556278078 (1.37 min)


## Interpretação

Como comentado, para além de conseguir encontrar valores numéricos precisos, a ideia é poder interpretar o resultado a partir da comparação dos valores. Um ponto interessante deve ser comentado, que é a pontuação baixa atribuída para questões fáceis e difíceis, porém uma pontuação maior para questões intermediárias. Podemos inferir que solucionar quesões de nível médio contribuiem mais para o atingimento de maestria do que questões extremamente fáceis ou difíceis.

Apesar do objetivo deste repositório não ser levantar hipóteses sobre a aprendizagem, podemos verificar numericamente que este comportamento de pontuação resulta em uma melhor acomodação da maestria dos estudantes. Assim, temos:

- Ganho por cada questão acertada: 4,44 pontos
- Ganho por cada questão nível 1 acertada: 0,33 pontos
- Ganho por cada questão nível 2 acertada: 0,86 pontos
- Ganho por cada questão nível 3 acertada: 0,86 pontos
- Ganho por cada questão nível 4 acertada: 0,46 pontos
- Ganho por cada questão nível 5 acertada: 0,39 pontos

In [22]:
best_f1, best_population = 0, []
for population, f1 in zip(current_population, current_results):
    if f1 > best_f1:
        best_f1 = f1
        best_population = population

print(f"Melhores parâmetros:")
print(best_population)

Melhores parâmetros:
[4.437959895811392, 0.3337814382791118, 0.8585858298354134, 0.8579571523067716, 0.46166699184186577, 0.38552925499586654]
